# Four on one row Agent

Lorem ipsum dolor sit amet, consectetur adipiscing elit. Duis vestibulum, nibh sit amet convallis varius, neque arcu luctus nunc, gravida sollicitudin quam nisi mattis ante. Duis imperdiet lorem a pharetra iaculis. In hac habitasse platea dictumst. Vivamus nisl ex, semper eget facilisis vel, varius ac est. Curabitur eget nisl erat. In egestas molestie mi, a pulvinar ante sodales sit amet. Vivamus interdum sem eu interdum placerat. Duis tempus purus non lectus pharetra, ac gravida mauris ultricies. Vestibulum condimentum ut dolor nec interdum. Curabitur et ex magna. In erat purus, laoreet vitae elementum sed, vehicula eu lorem. Nulla maximus eu est vel tempor. Ut et aliquam quam, eu pellentesque leo. Ut at suscipit eros. Phasellus luctus, justo nec molestie pharetra, massa sem hendrerit arcu, ut ullamcorper justo augue et est. Donec dictum sem id tristique pharetra.

#TODO state which libaries are used 

## Environment setup
... 

#TODO fix this!
This notebook defines an agent for the game four on one row, against 2 opponents. The first opponent randomly picks his actions, the second opponnent.... (still has to be defined).
The agent marks his spots using R, the opponent marks his stops using Y.

The game will be played on a field sized 5x5.

States: The environment has 25 states, S1 representing the top left spot and S25 representing the bottom right spot.
Actions:
Transitions:

- States: The Environment is in one of the following states $S = (S_0, S_1, ..., S_25)$ with $S_i \in \{E, R, Y\}$ (E represents an empty spot) .


- Actions: The set of available actions is $a \in \{n \in \mathbb{N}: S_n = E \land S_n-5 \neq  E  \}$ (the agent can place the X on any empty place with a surface)


- Transitions: 






In [1]:
from src import FourInARowEnv

In [2]:
environment = FourInARowEnv()

In [3]:
print(environment.render())

   │   │   │   │   │   │   
───┼───┼───┼───┼───┼───┼───
   │   │   │   │   │   │   
───┼───┼───┼───┼───┼───┼───
   │   │   │   │   │   │   
───┼───┼───┼───┼───┼───┼───
   │   │   │   │   │   │   
───┼───┼───┼───┼───┼───┼───
   │   │   │   │   │   │   
───┼───┼───┼───┼───┼───┼───
   │   │   │   │   │   │   


In [4]:
from enum import Enum
from random import randint, choice
from copy import copy

E, X, O = ' ', 'X', 'O'

class TicTacToeMDPEnvironment():
    def __init__(self, initial_state=None):
        if initial_state == None:
            self.__initial_state = [E for n in range(9)]  # start with empty board
        else:
            self.__initial_state = copy(initial_state)  # copy to prevent aliassing
        self.__state = self.__initial_state
        self.__possible_states = []
        self.__calculate_possible_states(self.__initial_state)
        
    def __calculate_possible_states(self, state):
        actions = self.get_possible_actions(state)
        for action in actions:
            new_state = copy(state)
            if state.count(X) == state.count(O):
                new_state[action] = X 
            else: 
                new_state[action] = O
            self.__possible_states.append(new_state)
            if not self.is_done(new_state):
                self.__calculate_possible_states(new_state)

    def reset(self):
        self.__state = self.__initial_state
        return self.__state
    
    def __calculate_transition(self, action):
        # change the state to reflect the move by the agent
        self.__state[action] = X
        if self.is_done():
            return self.__state
        # let the opponent make a random move
        opponent_action = choice(self.get_possible_actions())
        self.__state[opponent_action] = O
        return self.__state
      
    def step(self, action):
        old_state = self.__state
        self.__state = self.__calculate_transition(action)  # state after action
        observation = self.__state  # environment is fully observable
        done = self.is_done()
        reward = self.get_reward(self.__state)
        info = {}  # optional debug info
        return observation, done, reward, info

    def render(self):
        BACKGROUND = [
            '   │   │   ',
            '───┼───┼───',
            '   │   │   ',
            '───┼───┼───',
            '   │   │   '
        ]
        rendering = copy(BACKGROUND)
        for n, S_n in enumerate(self.__state):
            if S_n != E:
                row = 2 * (n // 3)
                col = 4 * (n % 3) + 1
                line = rendering[row]
                rendering[row] = line[:col] + S_n + line[col + 1:]
        
        for line in rendering:
            print(line)
        
    #=========================================================
    # public functions for agent to calculate optimal policy
    #=========================================================
    
    def get_possible_states(self):
        return self.__possible_states
    
    def get_possible_actions(self, state=None):
        if state is None:
            state = self.__state
        return [n for n in range(9) if state[n] == E]

    def is_done(self, state=None):
        if state is None:
            state = self.__state
        if E not in state:
            return True
        if state[0] == state[1] == state[2] != E:
            return True
        if state[3] == state[4] == state[5] != E:
            return True
        if state[6] == state[7] == state[8] != E:
            return True
        if state[0] == state[3] == state[6] != E:
            return True
        if state[1] == state[4] == state[7] != E:
            return True
        if state[2] == state[5] == state[8] != E:
            return True
        if state[0] == state[4] == state[8] != E:
            return True
        if state[2] == state[4] == state[6] != E:
            return True
        return False
    
    def get_reward(self, state):
        # Reward R(s) for every possible state
        if state[0] == state[1] == state[2] != E:
            return 1 if state[0] == X else -1
        if state[3] == state[4] == state[5] != E:
            return 1 if state[0] == X else -1
        if state[6] == state[7] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[0] == state[3] == state[6] != E:
            return 1 if state[0] == X else -1
        if state[1] == state[4] == state[7] != E:
            return 1 if state[0] == X else -1
        if state[2] == state[5] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[0] == state[4] == state[8] != E:
            return 1 if state[0] == X else -1
        if state[2] == state[4] == state[6] != E:
            return 1 if state[0] == X else -1
        return False
        
    def get_transition_prob(self, action, new_state, old_state=None):
        if old_state is None:
            old_state = self.__state
        # returns the Transition Probability P(s'| s, a)
        # with s = old_state, a = action and s' = new_state

        # if the game is over, no transition can take place
        if self.is_done(old_state):
            return 0.0
        
        # the position of the action must be empty
        if old_state[action] != E:
            return 0.0
        
        # state after placing X
        state_after_X = copy(old_state)  # avoid unwanted changed by reference
        state_after_X[action] = X

        # check if game is done
        if self.is_done(state_after_X) and state_after_X == new_state:
            return 1.0

        # game is not done: calculate all possible states of the opponent
        possible_new_states = []
        possible_opponent_actions = self.get_possible_actions(state_after_X)
        for action in possible_opponent_actions:
            possible_new_state = copy(state_after_X)
            possible_new_state[action] = O
            possible_new_states.append(possible_new_state)
        if new_state not in possible_new_states:
            return 0.0
        
        # transition is possible, apply strategy:
        # random opponent, probability is 1 / (# of E before placing the new O)
        prob = 1 / (len(possible_new_states))
        return prob
